In [1]:
import re
from typing import Any

In [2]:
def parse_expression(expr: str):
    """Parse expressions like $.foo == "some value" into field name and comparison value."""
    # Match pattern like $.fieldname == "value" or $.fieldname == 123
    pattern = r'\$\.([\w\.]+)\s*(==|!=|>|<|>=|<=)\s*(.+)'
    match = re.match(pattern, expr.strip())

    if not match:
        raise ValueError(f"Invalid expression format: {expr}")

    field, operator, value = match.groups()

    # Try to evaluate the value part (handles quoted strings, numbers, etc.)
    try:
        evaluated_value = eval(value)
    except:
        evaluated_value = value  # Keep as string if eval fails

    return field, operator, evaluated_value

In [3]:
test_expressions = [
    '$.foo == "some value"',
    '$.nested.field == 123',
    '$.active == True',
    '$.count > 5'
]

for expr in test_expressions:
    field, op, value = parse_expression(expr)
    print(f"Expression: {expr}")
    print(f"  Field: {field}")
    print(f"  Operator: {op}")
    print(f"  Value: {value} (type: {type(value).__name__})")
    print()

Expression: $.foo == "some value"
  Field: foo
  Operator: ==
  Value: some value (type: str)

Expression: $.nested.field == 123
  Field: nested.field
  Operator: ==
  Value: 123 (type: int)

Expression: $.active == True
  Field: active
  Operator: ==
  Value: True (type: bool)

Expression: $.count > 5
  Field: count
  Operator: >
  Value: 5 (type: int)



In [4]:
def evaluate_expression(expr: str, obj: dict[str, Any]) -> bool:
    """Evaluate expressions like $.foo == "some value" against an object."""
    # Check if pydash is installed
    try:
        import pydash
    except ImportError:
        !pip install pydash
        import pydash

    field, operator, value = parse_expression(expr)
    field_value = pydash.get(obj, field)

    if operator == "==":
        return field_value == value
    elif operator == "!=":
        return field_value != value
    elif operator == ">":
        return field_value > value
    elif operator == "<":
        return field_value < value
    elif operator == ">=":
        return field_value >= value
    elif operator == "<=":
        return field_value <= value
    else:
        raise ValueError(f"Unsupported operator: {operator}")

In [5]:
test_object = {
    "foo": "some value",
    "nested": {"field": 123},
    "active": True,
    "count": 10
}

for expr in test_expressions:
    result = evaluate_expression(expr, test_object)
    print(f"Expression: {expr}")
    print(f"  Result: {result

SyntaxError: incomplete input (525760041.py, line 11)

In [ ]:
class Expr:
    def __init__(self, args: str):
        self.args = args

    def test(self, o: dict[str, Any]) -> bool:
        """
        Evaluates expressions like '$.foo == "some value"' against the provided object.
        Also supports logical operators AND and OR for combining conditions:
        '$.foo == "value" AND $.count > 5'
        '$.status == "active" OR $.role == "admin"'

        The $ symbol represents the root of the object being tested.
        Supported comparison operators: ==, !=, >, <, >=, <=
        Supported logical operators: AND, OR (case insensitive)
        """
        # Check if pydash is installed
        try:
            import pydash
        except ImportError:
            !pip install pydash
            import pydash

        # Split on AND/OR (but not inside quotes)
        def split_on_logical_ops(expr):
            parts = []
            operators = []
            last_pos = 0
            in_quotes = False

            for i, char in enumerate(expr):
                if char in ['"', "'"]:
                    in_quotes = not in_quotes

                if not in_quotes and i + 5 <= len(expr):
                    if expr[i:i+5].upper() == ' AND ':
                        parts.append(expr[last_pos:i])
                        operators.append('AND')
                        last_pos = i + 5
                    elif expr[i:i+4].upper() == ' OR ':
                        parts.append(expr[last_pos:i])
                        operators.append('OR')
                        last_pos = i + 4

            parts.append(expr[last_pos:])
            return parts, operators

        # Evaluate a single comparison expression
        def evaluate_comparison(expr):
            # Parse expression: $.fieldpath operator value
            pattern = r'\$\.([\w\.]+)\s*(==|!=|>|<|>=|<=)\s*(.+)'
            match = re.match(pattern, expr.strip())

            if not match:
                raise ValueError(f"Invalid expression format: {expr}")

            field, operator, value = match.groups()

            # Try to evaluate the value part (handles quoted strings, numbers, etc.)
            try:
                evaluated_value = eval(value, {"__builtins__": {}}, {})
            except:
                evaluated_value = value  # Keep as string if eval fails

            # Get the field value from the object
            field_value = pydash.get(o, field)

            # Apply the operator
            if operator == "==":
                return field_value == evaluated_value
            elif operator == "!=":
                return field_value != evaluated_value
            elif operator == ">":
                return field_value > evaluated_value
            elif operator == "<":
                return field_value < evaluated_value
            elif operator == ">=":
                return field_value >= evaluated_value
            elif operator == "<=":
                return field_value <= evaluated_value
            else:
                raise ValueError(f"Unsupported operator: {operator}")

        # If there are no logical operators, evaluate as a simple expression
        if ' AND ' not in self.args.upper() and ' OR ' not in self.args.upper():
            return evaluate_comparison(self.args)

        # Split the expression by logical operators
        parts, operators = split_on_logical_ops(self.args)

        # Evaluate the first part
        result = evaluate_comparison(parts[0])

        # Apply each logical operator in sequence
        for i, op in enumerate(operators):
            if op == 'AND':
                result = result and evaluate_comparison(parts[i+1])
                # Short-circuit evaluation - if already False, no need to continue with AND
                if not result:
                    break
            elif op == 'OR':
                result = result or evaluate_comparison(parts[i+1])
                # Short-circuit evaluation - if already True, no need to continue with OR
                if result and (i+1 < len(operators) and operators[i+1] == 'OR'):
                    continue

        return result

In [ ]:
expr_instance = Expr('$.foo == "some value"')
result = expr_instance.test(test_object)
print(f"Expression: {expr_instance.args}")
print(f"Result: {result}")

expr_instance2 = Expr('$.count > 5')
result2 = expr_instance2.test(test_object)
print(f"Expression: {expr_instance2.args}")
print(f"Result: {result2}")

In [ ]:
test_object_extended = {
    "foo": "some value",
    "nested": {"field": 123},
    "active": True,
    "count": 10,
    "role": "user",
    "permissions": ["read", "write"]
}

print("\nTesting AND operator:")
expr_and = Expr('$.foo == "some value" AND $.count > 5')
result_and = expr_and.test(test_object_extended)
print(f"Expression: {expr_and.args}")
print(f"Result: {result_and}")  # Should be True

expr_and_false = Expr('$.foo == "some value" AND $.count < 5')
result_and_false = expr_and_false.test(test_object_extended)
print(f"Expression: {expr_and_false.args}")
print(f"Result: {result_and_false}")  # Should be False

print("\nTesting OR operator:")
expr_or = Expr('$.role == "admin" OR $.count > 5')
result_or = expr_or.test(test_object_extended)
print(f"Expression: {expr_or.args}")
print(f"Result: {result_or}")  # Should be True

expr_or_false = Expr('$.role == "admin" OR $.count < 5')
result_or_false = expr_or_false.test(test_object_extended)
print(f"Expression: {expr_or_false.args}")
print(f"Result: {result_or_false}")  # Should be False

print("\nTesting multiple operators:")
expr_complex = Expr('$.foo == "some value" AND $.role == "admin" OR $.count > 5')
result_complex = expr_complex.test(test_object_extended)
print(f"Expression: {expr_complex.args}")
print(f"Result: {result_complex}")  # Should be True